In [1]:
import pandas as pd
import random
import numpy as np
from scipy.spatial import distance_matrix
import itertools

In [2]:
def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)

    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm + temp + temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    if show:
        df_dist = pd.DataFrame(dm)
        # display(df_dist)
    return dm


def get_random_solution():
    return random.sample([i for i in range(0, 200)], 100)


def calulate_total_cost(path, dm):
    total = 0
    nr = len(path)
    for idx, node in enumerate(path):
        total += dm[node][path[(idx + 1) % nr]]
    return total

In [87]:
def generate_all_moves(path, LM, dm):
    new_nodes = list(set([i for i in range(0, 200)]) - set(path))
    
    #inter
    for idx, node in enumerate(path):
        for new_node in new_nodes:
            new_cost = dm[path[idx-1]][new_node] + dm[path[(idx+1)%100]][new_node]
            new_cost -= (dm[path[idx-1]][node] + dm[path[(idx+1)%100]][node])
            if new_cost < 0:
                old_move = [path[idx-1], path[idx], path[(idx+1)%100]]
                new_move = [path[idx-1], new_node, path[(idx+1)%100]]
                LM.append([new_cost, 'node', [old_move, new_move]])
    #intra
    for idx1, idx2 in all_posibble_comb:
        edge1, edge2 = [path[idx1], path[(idx1+1)%100]], [path[idx2], path[(idx2+1)%100]]
        if not(edge1[0] in edge2 or edge1[1] in edge2):
            new_cost = dm[edge1[0]][edge2[0]] + dm[edge1[1]][edge2[1]] 
            new_cost -= (dm[edge1[0]][edge1[1]] + dm[edge2[0]][edge2[1]])
            if new_cost < 0:
                LM.append([new_cost, 'edge_after', [edge1, edge2]])
        
        
    
    
    return LM

In [92]:
def check_move(path, move, typ):
    if typ == 'node':
        if move[0][1] in path and move[1][1] not in path: #if old nodes are in same order in path, and new node not in path
            pos0 = path.index(move[0][1])
            if path[pos0-1] == move[0][0] and path[(pos0+1)%100] == move[0][2]:
                return True, True
            if path[(pos0+1)%100] == move[0][0] and path[pos0-1] == move[0][2]:
                return True, True 
        
    elif typ ==  'edge_after':
        edge1 = move[0]
        edge2 = move[1]

        if edge1[0] in path and edge1[1] in path and edge2[0] in path and edge2[1] in path:
            n1, n2, x1, x2 = path.index(edge1[0]), path.index(edge1[1]), path.index(edge2[0]), path.index(edge2[1])
#             print('a', n1, n2, x1, x2)
            if (n2-n1 == 1 and x2-x1 == 1) or (n2-n1 == -1 and x2-x1 == -1):
                return True, True
            if (n2-n1 == -1 and x2-x1 == 1) or (n2-n1 == 1 and x2-x1 == -1):
                return False, True
    else:
#         print('d', move)
        edge1 = move[0]
        edge2 = move[1]

        if edge1[0] in path and edge1[1] in path and edge2[0] in path and edge2[1] in path:
            n1, n2, x1, x2 = path.index(edge1[0]), path.index(edge1[1]), path.index(edge2[0]), path.index(edge2[1])
#             print('b', n1, n2, x1, x2)
            if (n1-n2 == 1 and x1-x2 == 1) or (n1-n2 == -1 and x1-x2 == -1):
                return True, True
            if (n1-n2 == -1 and x1-x2 == 1) or (n1-n2 == 1 and x1-x2 == -1):
                return False, True
            
    return False, False


In [93]:
def new_moves_node(path, change_node_index, LM):
    new_nodes = list(set([i for i in range(0, 200)]) - set(path))
    for new_node in new_nodes:
        new_cost = dm[path[change_node_index-1]][new_node] + dm[path[(change_node_index+1)%100]][new_node]
        new_cost -= (dm[path[change_node_index-1]][path[change_node_index]] + dm[path[(change_node_index+1)%100]][path[change_node_index]])
#         print("new cost", new_cost)
        if new_cost < 0:
            old_move = [path[change_node_index-1], path[change_node_index], path[(change_node_index+1)%100]]
            new_move = [path[change_node_index-1], new_node, path[(change_node_index+1)%100]]
            LM.append([new_cost, 'node', [old_move, new_move]])
    return LM

In [94]:
def new_moves_edge_exchange(path, change_node_index, LM):
    for idx in range(len(path)):
        if idx != change_node_index:
            if idx > change_node_index:
                idx, change_node_index = change_node_index, idx
                
            edge1, edge2 = [path[idx], path[(idx+1)%100]], [path[change_node_index], path[(change_node_index+1)%100]]
            if not(edge1[0] in edge2 or edge1[1] in edge2):
                new_cost = dm[edge1[0]][edge2[0]] + dm[edge1[1]][edge2[1]] 
                new_cost -= (dm[edge1[0]][edge1[1]] + dm[edge2[0]][edge2[1]])
                if new_cost < 0:
                    LM.append([new_cost, 'edge_after', [edge1, edge2]])
            edge1, edge2 = [path[idx], path[idx-1]], [path[change_node_index], path[change_node_index-1]]
            if not(edge1[0] in edge2 or edge1[1] in edge2):
                new_cost = dm[edge1[0]][edge2[0]] + dm[edge1[1]][edge2[1]] 
                new_cost -= (dm[edge1[0]][edge1[1]] + dm[edge2[0]][edge2[1]])
                if new_cost < 0:
                    LM.append([new_cost, 'edge_before', [edge1, edge2]])
    return LM
#     pass


In [173]:
def a(path):
    LM = []
    LM = generate_all_moves(path, [], dm)
    # print(calulate_total_cost(path, dm))
    change = True
    i = 500
    while change:

        to_delete = []
        change = False
#         print('LM', len(LM))
        LM.sort(key=lambda x: x[0])
        if LM[0][0] > 0:
            break

        for cost_typ_move in LM:

            cost, typ, move = cost_typ_move
            good, keep = check_move(path, move, typ)
            if good:
                if typ == 'node':
                    idx = path.index(move[0][1])
                    path[idx] = move[1][1]
                    change = True
                    LM = new_moves_edge_exchange(path, idx, LM)
                    to_delete.append(cost_typ_move)
#                     print(typ, cost, calulate_total_cost(path, dm))   
                    break
                elif typ =='edge_after':
                    edge1, edge2 = move
                    n1, n2, x1, x2 = path.index(edge1[0]), path.index(edge1[1]), path.index(edge2[0]), path.index(edge2[1])
                    if n1 > x1:
                        n1, x1 = x1, n1 
                    path[n1: x1+1] = path[n1: x1+1][::-1]
                    change = True
                    LM = new_moves_node(path, n1, LM)
                    LM = new_moves_node(path, x1, LM)

#                     print(typ, cost, calulate_total_cost(path, dm))
                    to_delete.append(cost_typ_move)
                    break
                else:
                    edge1, edge2 = move
                    n1, n2, x1, x2 = path.index(edge1[0]), path.index(edge1[1]), path.index(edge2[0]), path.index(edge2[1])
                    if n1 > x1:
                        n1, x1 = x1, n1 
                    path[n1: x1] = path[n1: x1][::-1]
                    change = True
                    LM = new_moves_node(path, n1, LM)
                    LM = new_moves_node(path, x1, LM)

#                     print(typ, cost, calulate_total_cost(path, dm))
                    to_delete.append(cost_typ_move)
                    break


            if not keep:
                to_delete.append(cost_typ_move)

        for cost_typ_move in to_delete:
            LM.remove(cost_typ_move)
        i+=1
        if i == 500: 
            break
#     print(i)

    return calulate_total_cost(path, dm), path      
path = get_random_solution()
w = a(path)
w[0], len(w[1])

(67419.0, 100)

In [174]:
from tqdm import tqdm

In [175]:
all_posibble_comb = list(itertools.combinations([i for i in range(100)], 2))
tsp = pd.read_csv('TSPC.csv', sep=';', header=None)
dm = generate_dm(tsp)
costs = []
paths = []
for i in tqdm(range(200)):
    path = get_random_solution()
    cost, path = a(path)
    costs.append(cost)
    paths.append(path)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:40<00:00,  4.91it/s]


In [178]:
np.array(costs).mean()
# import copy
# path2 = copy.deepcopy(path)
# path2[idx1:idx2+1] = path2[idx1:idx2+1][::-1]
# path == path2

67106.555